In [1]:
import matplotlib.pyplot as plt
from matplotlib import cm
from ipywidgets import interactive, Layout, HTML, HBox, VBox, FloatSlider
import numpy as np
import warnings

warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'

**Generalised logistic function (GLF)**

![y = A + \frac{K - A}{\left( C + Q \cdot e^{-By} \right)^{ 1 / \nu} }](assets/GLF.png)

taken from https://en.wikipedia.org/wiki/Generalised_logistic_function

**Inverse GLF**
The following equation is the one used in this simulation.

![y = \frac{1}{B} \cdot ln \left( \frac{ \left( \frac{K - A}{x - A} \right)^\nu -C }{Q} \right) + E](assets/GLF_inverse.png)

A bunch of parameter can be removed, as they are not used:
- `A` is the minimum value of the x axis, which will always be `0`
- `C` and `Q` are form parameters that are meant to be set to `1` by default

With those parameters replaced by their default values, here is the simplified equation used:

![y = \frac{1}{B} \cdot ln \left( \left( \frac{K}{x} \right)^\nu -1 \right) + E](assets/GLF_inverse_simplified.png)

This equation can be generalised in other dimensions as follow:
 

In [105]:
class Param:
    def __init__(self, name, value, min, max, step, description, group="default"):
        self.name = name
        self.value = value
        self.min = min
        self.max = max
        self.step = step
        self.description = description
        self.group = group

    def slider(self):
        style = {'description_width': 'initial', 'value_width': 'initial'}

        return FloatSlider(min=self.min, max=self.max, step=self.step, value=self.value, description=f"{self.description} ({self.name})", style=style)
        
def equation(X, Y, E, B1, K1, nu1, B2, K2, nu2):
    try:
        inner1 = np.power((K1/ X), nu1) - 1
        inner2 = np.power((K2/ Y), nu2) - 1

        output: np.ndarray = (1/B1) * np.log(inner1) + (1/B2) * np.log(inner2) + E
    except ZeroDivisionError:
        output: np.ndarray = np.zeros_like(X)*np.nan

    return output

def plot_static(E, B1, K1, nu1, B2, K2, nu2, alpha, theta):
    ranges = [
        np.arange(0, 1, 0.001),
        np.arange(0, 1, 0.001)
    ]
    grid = np.meshgrid(*ranges)
    T = equation(*grid, E, B1, K1, nu1, B2, K2, nu2)
    
    fig, ax = plt.subplots(subplot_kw={'projection': '3d'}, figsize=(10, 10))
    fig.tight_layout(pad=5)
    
    ax.plot_surface(*grid, T, linewidth=0.5, cmap=cm.viridis, rstride=20, cstride=20)#,  vmin=0, vmax=20)
    ax.contour(*grid, T, zdir='z', offset=0, cmap='coolwarm')
    ax.contour(*grid, T, zdir='x', offset=-.5, cmap='coolwarm')
    ax.contour(*grid, T, zdir='y', offset=-.5, cmap='coolwarm')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('APR (%)')

    ax.set_xlim(-.5, 1)
    ax.set_ylim(-.5, 1)
    ax.set_zlim(0, 20)
    ax.view_init(alpha, theta)

    # ax.tick_params(axis="z", pad=-3) # (+) Added


def plot_interactive(params: list[Param]):
    sliders_groups = {item.name: item.group for item in params}
    sliders_list = {item.name: item.slider() for item in params}

    widgets = interactive(plot_static, **sliders_list)
    
    grouped_widgets = {group: [] for group in set([item.group for item in params])}
    for name, widget in zip(sliders_list.keys(), widgets.children[:-1]):
        grouped_widgets[sliders_groups[name]].append(widget)

    widget_title_group = [
        VBox([
            HTML(value=f"<b>{name}</b>"), 
            HBox(widget_group, layout = Layout(flex_flow='row wrap')), 
            HTML(value="<br>")
        ])
        for name, widget_group in grouped_widgets.items()
    ]

    group_names = grouped_widgets.keys()
    ordered_group = [x for _, x in sorted(zip(group_names, widget_title_group))]

    display(VBox(ordered_group))
    display(widgets.children[-1])

In [106]:
params = [
    Param("E", 11.5, 0, 20, 0.1, "V. Offset", "Global"),

    Param("B1", 1.25, 0, 10, 0.01, "Flatness", "Curve #1"),
    Param("K1", 0.5, 0, 1, 0.01, "Maximum ratio", "Curve #1"),
    Param("nu1",1.5, 0, 2, 0.01, "Skewness", "Curve #1"),

    Param("B2", 1.25, 0, 10, 0.01, "Flatness", "Curve #2"),
    Param("K2", 0.5, 0, 1, 0.01, "Maximum ratio", "Curve #2"),
    Param("nu2",1.5, 0, 2, 0.01, "Skewness", "Curve #2"),

    Param("alpha", 30, 0, 90, 5, "Alpha", "UI"),
    Param("theta", 45, 0, 90, 5, "Theta", "UI"),
]

plot_interactive( params)

Output()